In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)
tqdm.tqdm.pandas()

In [2]:
%%time
main_data_df = pd.read_csv('./Prepped Data/main_data_coor_days_nonan_20211224.csv', dtype={1:'str', 9:'Int64', 10:'Int64'})
main_data_df['계약날짜'] = pd.to_datetime(main_data_df['계약날짜'])
print(main_data_df.shape)
main_data_df.head()

(5068169, 22)
Wall time: 13.3 s


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준
0,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,2253,12,12,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-02-22,4070
1,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,730,10,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-04-26,4133
2,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,1815,4,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-08-31,4260
3,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,3865,17,11,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2018-03-29,4470
4,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,3062,7,9,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2018-07-14,4577


In [3]:
main_data_df['구'] = main_data_df['시군구'].apply(lambda x: x.split(' ')[1])
main_data_df['동'] = main_data_df['시군구'].apply(lambda x: x.split(' ')[2])

In [4]:
main_data_df['계약날짜일수_2006년기준'] = main_data_df['계약날짜일수_2006년기준'].astype('Int64')

In [5]:
main_data_df.isna().sum()

시군구               0
번지                0
본번                0
부번                0
건물/단지명            0
거래유형              0
전용면적(㎡)           0
거래/보증금(만원)        0
월세(만원)            0
층                 0
건축년도              0
도로명               0
부동산유형             0
가격범위              0
건물단지명_키워드         0
브랜드               0
대장구분명             0
지목                0
x좌표               0
y좌표               0
계약날짜              0
계약날짜일수_2006년기준    0
구                 0
동                 0
dtype: int64

In [6]:
main_data_df['주소_건축년도'] = main_data_df['시군구'] + ' ' + main_data_df['번지'].astype('str') + ' ' + main_data_df['건축년도'].astype('str')

In [7]:
def floor_feature_engi(x):
    # x is a number
    
    if x < 0:
        return '지하'
    elif x > 40:
        return '40층초과'
    else:
        return str(x)

In [8]:
main_data_df['층'] = main_data_df['층'].apply(floor_feature_engi)

In [9]:
main_data_df['층'].value_counts()

2        665030
3        641389
4        552041
5        420713
1        387793
6        275198
7        238755
8        219686
9        209668
10       199348
11       182661
12       173596
13       152158
14       140409
15       124095
지하        79984
16        66901
17        61442
18        54680
19        45135
20        37796
21        27165
22        23097
23        18993
24        14873
25        11822
26         7628
27         6426
28         5181
29         4191
30         3235
31         2637
40층초과      2592
32         2256
33         1936
0          1639
34         1637
35         1352
36          972
37          646
38          583
39          487
40          343
Name: 층, dtype: int64

In [11]:
inv_df = main_data_df[(main_data_df['거래유형'] != '월세') & (main_data_df['월세(만원)'] != 0)]
print(inv_df.shape)
inv_df.head()

(17, 25)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,구,동,주소_건축년도
189131,서울특별시 강남구 삼성동,141-10,141,10,삼성동 롯데캐슬 클라쎄,전세,21.87,24000,3,6,2019,테헤란로63길 9,오피스텔,22000~24000,삼성동 롯데캐슬 클라쎄,롯데캐슬,일반,대,204369.880744,444844.447272,2021-08-17,5707,강남구,삼성동,서울특별시 강남구 삼성동 141-10 2019
804116,서울특별시 강서구 등촌동,631,631,0,등촌동두산위브센티움,전세,43.11,5000,60,3,2004,양천로 564,오피스텔,1억4천 이하,등촌동두산위브센티움,기타,일반,대,187682.690364,450578.892836,2021-06-21,5650,강서구,등촌동,서울특별시 강서구 등촌동 631 2004
880707,서울특별시 강서구 마곡동,795-2,795,2,마곡아이파크,전세,36.98,2000,82,7,2016,마곡중앙8로 90,오피스텔,1억4천 이하,마곡아이파크,아이파크,일반,대,185590.593750,450933.823750,2021-07-27,5686,강서구,마곡동,서울특별시 강서구 마곡동 795-2 2016
881102,서울특별시 강서구 마곡동,795-5,795,5,마곡사이언스파크뷰,전세,30.56,20000,2,11,2016,마곡중앙6로 85,오피스텔,18000~20000,마곡사이언스파크뷰,기타,일반,대,185517.525000,450897.540000,2021-06-29,5658,강서구,마곡동,서울특별시 강서구 마곡동 795-5 2016
1062895,서울특별시 강서구 화곡동,982-17,982,17,크라운뷰,전세,35.54,2000,50,5,2009,화곡로 274,오피스텔,1억4천 이하,크라운뷰,기타,일반,대,186457.158547,449646.201516,2021-09-08,5729,강서구,화곡동,서울특별시 강서구 화곡동 982-17 2009


In [12]:
main_data_df.loc[inv_df.index, '거래유형'] = '월세'

In [13]:
negative_df = main_data_df[
    (main_data_df['거래/보증금(만원)'] < 0)
    |(main_data_df['월세(만원)'] < 0)
]

In [14]:
print(negative_df.shape)
negative_df.head()

(0, 25)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,구,동,주소_건축년도


In [15]:
main_data_df['계약날짜기준_건물연식'] = main_data_df['계약날짜'].dt.year - main_data_df['건축년도']

In [16]:
def year_circular_sine_func(x):
    return np.sin(2*np.pi*((x)/365))

def year_circular_cosine_func(x):
    return np.cos(2*np.pi*((x)/365))

In [17]:
main_data_df['계약날짜일수_sin'] = main_data_df['계약날짜일수_2006년기준'].apply(year_circular_sine_func)
main_data_df['계약날짜일수_cos'] = main_data_df['계약날짜일수_2006년기준'].apply(year_circular_cosine_func)

In [18]:
main_data_df.head()

,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,구,동,주소_건축년도,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos
0,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,2253,12,12,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-02-22,4070,강남구,개포동,서울특별시 강남구 개포동 12 1991,26,0.811539,0.584298
1,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,730,10,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-04-26,4133,강남구,개포동,서울특별시 강남구 개포동 12 1991,26,0.895839,-0.444378
2,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,1815,4,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-08-31,4260,강남구,개포동,서울특별시 강남구 개포동 12 1991,26,-0.880012,-0.474951
3,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,3865,17,11,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2018-03-29,4470,강남구,개포동,서울특별시 강남구 개포동 12 1991,27,0.999769,0.021516
4,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,3062,7,9,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2018-07-14,4577,강남구,개포동,서울특별시 강남구 개포동 12 1991,27,-0.247022,-0.969010


In [19]:
main_data_df['지목'].value_counts()

대    5043737
임       5555
전       5065
도       4965
답       2764
장       2039
잡       1690
철        898
차        512
종        347
주        300
공        119
학         67
수         46
창         32
과         20
사         13
Name: 지목, dtype: int64

In [20]:
main_data_df.dtypes

시군구                       object
번지                        object
본번                         int64
부번                         int64
건물/단지명                    object
거래유형                      object
전용면적(㎡)                  float64
거래/보증금(만원)                 int64
월세(만원)                     int64
층                         object
건축년도                       Int64
도로명                       object
부동산유형                     object
가격범위                      object
건물단지명_키워드                 object
브랜드                       object
대장구분명                     object
지목                        object
x좌표                      float64
y좌표                      float64
계약날짜              datetime64[ns]
계약날짜일수_2006년기준             Int64
구                         object
동                         object
주소_건축년도                   object
계약날짜기준_건물연식                Int64
계약날짜일수_sin               float64
계약날짜일수_cos               float64
dtype: object

In [23]:
cat_cols = ['거래유형', '층', '부동산유형', '브랜드', '대장구분명', '지목', '구', '동']

In [24]:
for col in tqdm.tqdm_notebook(cat_cols):
    unique_cats = main_data_df[col].unique()
    
    # mapping
    cat_map = {}
    for i, v in enumerate(unique_cats):
        cat_map[v] = i
        
    main_data_df[col] = main_data_df[col].map(cat_map)
    
    map_df = pd.DataFrame(cat_map, index=[0])
    map_df.to_excel('./Training/Training3_20220125/cat_col_maps/{}'.format(col+'.xlsx'), index=False)

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/8 [00:00<?, ?it/s]

In [25]:
training_data_df = main_data_df[[
    '주소_건축년도', '건축년도', '전용면적(㎡)', '월세(만원)', 'x좌표', 'y좌표', '계약날짜일수_2006년기준', '계약날짜기준_건물연식',
    '계약날짜일수_sin', '계약날짜일수_cos'
] + cat_cols + ['거래/보증금(만원)']]
print(training_data_df.shape)
training_data_df.head()

(5068169, 19)


,주소_건축년도,건축년도,전용면적(㎡),월세(만원),x좌표,y좌표,계약날짜일수_2006년기준,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,거래유형,층,부동산유형,브랜드,대장구분명,지목,구,동,거래/보증금(만원)
0,서울특별시 강남구 개포동 12 1991,1991,25.35,12,206607.181143,443586.061143,4070,26,0.811539,0.584298,0,0,0,0,0,0,0,0,2253
1,서울특별시 강남구 개포동 12 1991,1991,25.35,10,206607.181143,443586.061143,4133,26,0.895839,-0.444378,0,1,0,0,0,0,0,0,730
2,서울특별시 강남구 개포동 12 1991,1991,25.35,4,206607.181143,443586.061143,4260,26,-0.880012,-0.474951,0,1,0,0,0,0,0,0,1815
3,서울특별시 강남구 개포동 12 1991,1991,25.35,17,206607.181143,443586.061143,4470,27,0.999769,0.021516,0,2,0,0,0,0,0,0,3865
4,서울특별시 강남구 개포동 12 1991,1991,25.35,7,206607.181143,443586.061143,4577,27,-0.247022,-0.969010,0,3,0,0,0,0,0,0,3062


In [26]:
training_data_df['x좌표'] = training_data_df['x좌표'] / 100000
training_data_df['y좌표'] = training_data_df['y좌표'] / 100000

In [27]:
training_data_df.head()

,주소_건축년도,건축년도,전용면적(㎡),월세(만원),x좌표,y좌표,계약날짜일수_2006년기준,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,거래유형,층,부동산유형,브랜드,대장구분명,지목,구,동,거래/보증금(만원)
0,서울특별시 강남구 개포동 12 1991,1991,25.35,12,2.066072,4.435861,4070,26,0.811539,0.584298,0,0,0,0,0,0,0,0,2253
1,서울특별시 강남구 개포동 12 1991,1991,25.35,10,2.066072,4.435861,4133,26,0.895839,-0.444378,0,1,0,0,0,0,0,0,730
2,서울특별시 강남구 개포동 12 1991,1991,25.35,4,2.066072,4.435861,4260,26,-0.880012,-0.474951,0,1,0,0,0,0,0,0,1815
3,서울특별시 강남구 개포동 12 1991,1991,25.35,17,2.066072,4.435861,4470,27,0.999769,0.021516,0,2,0,0,0,0,0,0,3865
4,서울특별시 강남구 개포동 12 1991,1991,25.35,7,2.066072,4.435861,4577,27,-0.247022,-0.969010,0,3,0,0,0,0,0,0,3062


In [28]:
%%time
training_data_df.to_csv('./Prepped Data/training_data_20220125.csv', index=False)

Wall time: 26.9 s
